# RFP Response Generation Workflow (with Human-in-the-Loop)

<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/report_generation/rfp_response/generate_rfp_hitl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how to build a workflow to generate a response to an RFP. 

In this scenario, we assume that you are Microsoft, and you are responding to the [JEDI Cloud RFP](https://imlive.s3.amazonaws.com/Federal%20Government/ID151830346965529215587195222610265670631/HQ0034-18-R-0077.pdf) put out by the federal government. The government is using the submitted responses to decide the best vendor for their needs.

![generate_rfp_img](generate_rfp_hitl_img.png)

We index a set of relevant documents that Microsoft has - including its annual report, wikipedia page on Microsoft Azure, a slide deck on the government cloud and cybersecurity capabilities. We then help you build an agentic workflow that can ingest an RFP, and generate a response for it in 
a way that adheres to its guidelines.

We use LlamaCloud to index the documents and get back a set of retrieval endpoints over the documents. This tutorial takes full advantage of LlamaCloud as an e2e RAG platform. If you want to check out a similar tutorial that uses the LlamaParse API, check this other [tutorial out instead](https://github.com/run-llama/llama_parse/blob/main/examples/report_generation/rfp_response/generate_rfp.ipynb).

In [ ]:
!pip install llama-index llama-index-indices-llama-cloud llama-cloud llama-parse 

In [1]:
import nest_asyncio

nest_asyncio.apply()

## Setup LlamaCloud Index

We download the context documents for Microsoft to form the knowledge base.
1. Microsoft 2024 10-K 
2. Azure Wikipedia page
3. A slide deck on Microsoft Azure Government
4. Microsoft Digital Defense Report

In [ ]:
# microsoft annual report
!wget "https://www.dropbox.com/scl/fi/4v5dx8dc9yqc8k0yw5g4h/msft_10k_2024.pdf?rlkey=jdyfrsoyb18ztlq5msunmibns&st=9w6bdyvn&dl=1" -O data/msft_10k_2024.pdf
# !wget "https://microsoft.gcs-web.com/static-files/1c864583-06f7-40cc-a94d-d11400c83cc8" -O data/msft_10k_2024.pdf

# azure wikipedia page
!wget "https://www.dropbox.com/scl/fi/7waur8ravmve3fe8nej0k/azure_wiki.pdf?rlkey=icru2w64oylx1p76ftt6y9irv&st=fr87vxob&dl=1" -O data/azure_wiki.pdf
# azure government slide deck
!wget "https://cdn.ymaws.com/flclerks.site-ym.com/resource/resmgr/2017_Fall_Conf/Presentations/2018-10-12_FCCC_Microsoft_Az.pdf" -O data/azure_gov.pdf
# microsoft cybersecurity capabilities
!wget "https://www.dropbox.com/scl/fi/qh00xz29rlom4md8ce675/microsoft_ddr.pdf?rlkey=d868nbnsu1ng41y1chw69y64b&st=24iqemb1&dl=1" -O data/msft_ddr.pdf

We then upload these documents to LlamaCloud. For best results:
- Use "3rd Party multi-modal model" in the Parse Settings
- Use Page-level segmentation and "None" for chunking configuration

We then use a utility function to generate retriever tools from this index. If you want a step-by-step walkthrough on how this is generated, check out our [initial LlamaCloud example of RFP generation](https://github.com/run-llama/llamacloud-demo/blob/main/examples/report_generation/rfp_response/generate_rfp.ipynb). 

In [2]:
from generate_rfp_utils import generate_tools
import os

data_out_dir = "data_out_rfp_hitl"

tools = generate_tools(
    index_name="msft_rfp_data_2", 
    index_id="<index_id>",
    project_name="<project_name>",
    organization_id="<organization_id>",
    api_key=os.environ["LLAMA_CLOUD_API_KEY"],
    data_out_dir=data_out_dir
)

In [3]:
# validate an existing function
tools[0].metadata

ToolMetadata(description='    Synthesizes an answer to your question by feeding in relevant chunks of a document as context. Best used for questions that are more pointed in nature.\n    Do NOT use if the question asks seems to require a general summary of any given document. Use the doc_query_engine instead for that purpose.\n    \n    Document: msft_ddr.pdf\n    \n\nFile Description: The Microsoft Digital Defense Report, published in October 2023, provides insights into the evolving cyber threat landscape from July 2022 to June 2023, highlighting key developments in cybercrime, nation-state threats, and the importance of collaboration in enhancing global cybersecurity resilience.', name='msft_ddr_retrieve', fn_schema=<class 'llama_index.core.tools.utils.msft_ddr_retrieve'>, return_direct=False)

## Build Workflow

Let's build a workflow that can iterate through the extracted keys/questions from the RFP, and fill them out! 

The user specifies an RFP document as input. The workflow then goes through the following steps:
1. We parse the RFP template using LlamaParse
2. We then extract out the relevant questions we'd want to ask the knowledge base given the instructions in the RFP
3. For each question, we query the knowledge base using a specialized agent to generate a response. The agent is equipped with a set of retrieval tools over the data.
4. We concatenate the questions/answers into a list of dictionaries.
5. Given the question/answer pairs, we feed it along with the source RFP template into a prompt to generate the final report.



We download the [JEDI RFP template](https://imlive.s3.amazonaws.com/Federal%20Government/ID151830346965529215587195222610265670631/HQ0034-18-R-0077.pdf).

In [ ]:
# download JEDI Cloud RFP Template
!mkdir -p data
!wget "https://imlive.s3.amazonaws.com/Federal%20Government/ID151830346965529215587195222610265670631/HQ0034-18-R-0077.pdf" -O data/jedi_cloud_rfp.pdf

In [4]:
from llama_parse import LlamaParse

# use our multimodal models for extractions
parser = LlamaParse(result_type="markdown")

In [43]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Context,
    Workflow,
    step,
)
from llama_index.core.llms import LLM
from typing import Optional, List
from pydantic import BaseModel, Field
from llama_index.core.schema import Document
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.prompts import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.workflow.events import InputRequiredEvent, HumanResponseEvent
import logging
import json
import os
from pathlib import Path

_logger = logging.getLogger(__name__)
_logger.setLevel(logging.INFO)


# this is the research agent's system prompt, tasked with answering a specific question
AGENT_SYSTEM_PROMPT = """\
You are a research agent tasked with filling out a specific form key/question with the appropriate value, given a bank of context.
You are given a specific form key/question. Think step-by-step and use the existing set of tools to help answer the question.

You MUST always use at least one tool to answer each question. Only after you've determined that existing tools do not \
answer the question should you try to reason from first principles and prior knowledge to answer the question.

You MUST try to answer the question instead of only saying 'I dont know'.

"""

# This is the prompt tasked with extracting information from an RFP file.
EXTRACT_KEYS_PROMPT = """\
You are provided an entire RFP document, or a large subsection from it. 

We wish to generate a response to the RFP in a way that adheres to the instructions within the RFP, \
including the specific sections that an RFP response should contain, and the content that would need to go \
into each section.

Your task is to extract out a list of "questions", where each question corresponds to a specific section that is required in the RFP response.
Put another way, after we extract out the questions we will go through each question and answer each one \
with our downstream research assistant, and the combined
question:answer pairs will constitute the full RFP response.

You must TRY to extract out questions that can be answered by the provided knowledge base. We provide the list of file metadata below. 

Additional requirements:
- Try to make the questions SPECIFIC given your knowledge of the RFP and the knowledge base. Instead of asking a question like \
"How do we ensure security" ask a question that actually addresses a security requirement in the RFP and can be addressed by the knowledge base.
- Make sure the questions are comprehensive and addresses all the RFP requirements.
- Make sure each question is descriptive - this gives our downstream assistant context to fill out the value for that question 
- Extract out all the questions as a list of strings.


Knowledge Base Files:
{file_metadata}

RFP Full Template:
{rfp_text}

"""


# This is the prompt that takes in human input and decides whether to continue or stop 
REFLECT_ON_QA_PROMPT = """\
You are an agent that is provided with a question, an attempt to answer it, and then human feedback on whether the answer is correct.
There is a separate research agent that attempted to generate an answer to the question.
Given the human feedback, your goal is to decide between two choices - whether to finalize the answer, or pass it back to the research agent for refinement.
If answer is not finalized, then output a new question to ask the research agent.
Output a boolean in accordance with the provided schema.

Question: {question}
Answer: {answer}
Human Feedback: {human_feedback}

"""

class ReflectQAChoice(BaseModel):
    should_finalize: bool = Field(..., description="Whether to finalize the answer or not.")
    new_question: Optional[str] = Field(default=None, description="If answer is not finalized, suggest a new question to ask.")
    reasoning: str = Field(..., description="Provide reasoning for if answer is finalized or not, and for the new question to ask if not")


# this is the prompt that generates the final RFP response given the original template text and question-answer pairs.
GENERATE_OUTPUT_PROMPT = """\
You are an expert analyst.
Your task is to generate an RFP response according to the given RFP and question/answer pairs.

You are given the following RFP and qa pairs:

<rfp_document>
{output_template}
</rfp_document>

<question_answer_pairs>
{answers}
</question_answer_pairs>

Not every question has an appropriate answer. This is because the agent tasked with answering the question did not have the right context to answer it.
If this is the case, you MUST come up with an answer that is reasonable. You CANNOT say that you are unsure in any area of the RFP response.


Please generate the output according to the template and the answers, in markdown format.
Directly output the generated markdown content, do not add any additional text, such as "```markdown" or "Here is the output:".
Follow the original format of the template as closely as possible, and fill in the answers into the appropriate sections.
"""


class OutputQuestions(BaseModel):
    """List of keys that make up the sections of the RFP response."""

    questions: List[str]


class OutputTemplateEvent(Event):
    docs: List[Document]


class QuestionsExtractedEvent(Event):
    questions: List[str]


class HandleQuestionEvent(Event):
    question: str
    prev_history: List[ChatMessage] = Field(default_factory=list)

class QuestionAnsweredEvent(Event):
    question: str
    answer: str

class CollectedAnswersEvent(Event):
    combined_answers: str


class LogEvent(Event):
    msg: str
    delta: bool = False


class RFPWorkflow(Workflow):
    """RFP workflow."""

    def __init__(
        self,
        tools,
        parser: LlamaParse,
        llm: LLM | None = None,
        similarity_top_k: int = 20,
        output_dir: str = data_out_dir,
        agent_system_prompt: str = AGENT_SYSTEM_PROMPT,
        reflect_on_qa_prompt: str = REFLECT_ON_QA_PROMPT,
        generate_output_prompt: str = GENERATE_OUTPUT_PROMPT,
        extract_keys_prompt: str = EXTRACT_KEYS_PROMPT,
        **kwargs,
    ) -> None:
        """Init params."""
        super().__init__(**kwargs)
        self.tools = tools

        self.parser = parser

        self.llm = llm or OpenAI(model="gpt-4o-mini")
        self.similarity_top_k = similarity_top_k

        self.output_dir = output_dir

        self.agent_system_prompt = agent_system_prompt
        self.extract_keys_prompt = extract_keys_prompt
        self.reflect_on_qa_prompt = reflect_on_qa_prompt

        # if not exists, create
        out_path = Path(self.output_dir) / "workflow_output"
        if not out_path.exists():
            out_path.mkdir(parents=True, exist_ok=True)
            os.chmod(str(out_path), 0o0777)

        self.generate_output_prompt = PromptTemplate(generate_output_prompt)

    @step
    async def parse_output_template(
        self, ctx: Context, ev: StartEvent
    ) -> OutputTemplateEvent:
        # load output template file
        out_template_path = Path(
            f"{self.output_dir}/workflow_output/output_template.jsonl"
        )
        if out_template_path.exists():
            with open(out_template_path, "r") as f:
                docs = [Document.model_validate_json(line) for line in f]
        else:
            docs = await self.parser.aload_data(ev.rfp_template_path)
            # save output template to file
            with open(out_template_path, "w") as f:
                for doc in docs:
                    f.write(doc.model_dump_json())
                    f.write("\n")

        await ctx.set("output_template", docs)
        return OutputTemplateEvent(docs=docs)

    @step
    async def extract_questions(
        self, ctx: Context, ev: OutputTemplateEvent
    ) -> HandleQuestionEvent:
        docs = ev.docs

        # save all_questions to file
        out_keys_path = Path(f"{self.output_dir}/workflow_output/all_keys.txt")
        if out_keys_path.exists():
            with open(out_keys_path, "r") as f:
                output_qs = [q.strip() for q in f.readlines()]
        else:
            # try stuffing all text into the prompt
            all_text = "\n\n".join([d.get_content(metadata_mode="all") for d in docs])
            prompt = PromptTemplate(template=self.extract_keys_prompt)

            file_metadata = "\n\n".join([f"Name:{t.metadata.name}\nDescription:{t.metadata.description}" for t in tools])
            try:
                if self._verbose:
                    ctx.write_event_to_stream(LogEvent(msg=">> Extracting questions from LLM"))
                
                output_qs = self.llm.structured_predict(
                    OutputQuestions, 
                    prompt, 
                    file_metadata=file_metadata,
                    rfp_text=all_text,
                ).questions

                if self._verbose:
                    qs_text = "\n".join([f"* {q}" for q in output_qs])
                    ctx.write_event_to_stream(LogEvent(msg=f">> Questions:\n{qs_text}"))
            
            except Exception as e:
                _logger.error(f"Error extracting questions from page: {all_text}")
                _logger.error(e)

            with open(out_keys_path, "w") as f:
                f.write("\n".join(output_qs))

        await ctx.set("num_to_collect", len(output_qs))

        for question in output_qs:
            ctx.send_event(HandleQuestionEvent(question=question))

        return None

    @step
    async def handle_question(
        self, ctx: Context, ev: HandleQuestionEvent
    ) -> InputRequiredEvent:
        question = ev.question

        # initialize a Function Calling "research" agent where given a task, it can pull responses from relevant tools and synthesize over it
        chat_history = ev.prev_history

        if self._verbose and chat_history:
            ctx.write_event_to_stream(LogEvent(msg=f">> Existing chat history: {chat_history}"))


        prefix_messages = [
            ChatMessage.from_str(self.agent_system_prompt, "system"),
            *chat_history
        ]
        
        research_agent = FunctionCallingAgentWorker.from_tools(
            tools, 
            llm=llm, 
            verbose=False, 
            # system_prompt=self.agent_system_prompt,
            prefix_messages=prefix_messages
        ).as_agent()

        # ensure the agent's memory is cleared
        response = await research_agent.aquery(question)

        if self._verbose:
            # instead of printing the message directly, write the event to stream!
            msg = f">> Asked question: {question}\n>> Got response: {str(response)}"
            ctx.write_event_to_stream(LogEvent(msg=msg))

        # ask human to verify question/answer
        return InputRequiredEvent(
            prefix="Do you like this answer? ", 
            question=question, 
            answer=str(response),
            prev_history=ev.prev_history
        )

    @step
    async def process_human_response(
        self, ctx: Context, ev: HumanResponseEvent
    ) -> HandleQuestionEvent | QuestionAnsweredEvent:
        """Process human input - decide to pass it back to the agent or move ahead."""
        if self._verbose:
            msg = f""">> Processing human response:
Feedback: {ev.response}
Original question: {ev.question}
Original answer: {ev.answer}"""
            ctx.write_event_to_stream(LogEvent(msg=msg))

        prompt = PromptTemplate(template=self.reflect_on_qa_prompt)
        reflect_choice = self.llm.structured_predict(
            ReflectQAChoice,
            prompt,
            question=ev.question,
            answer=ev.answer,
            human_feedback=ev.response
        )
        if reflect_choice.should_finalize:
            if self._verbose:
                msg = ">> Finalized."
                ctx.write_event_to_stream(LogEvent(msg=msg))
            return QuestionAnsweredEvent(question=ev.question, answer=str(ev.answer))
        else:
            new_question = reflect_choice.new_question if reflect_choice.new_question else ev.question

            if self._verbose:
                msg = f"""\
>>Answer is incomplete, passing task back to research agent.
Reasoning: {reflect_choice.reasoning}
New question: {new_question}"""
                
                ctx.write_event_to_stream(LogEvent(msg=msg))
            
            # attach existing messages to the history
            prev_history = [
                *ev.prev_history,
                ChatMessage.from_str(ev.question, "user"),
                ChatMessage.from_str(ev.answer, "assistant"),
                ChatMessage.from_str(ev.response, "user")
            ]
            
            return HandleQuestionEvent(question=new_question, prev_history=prev_history)

    @step
    async def combine_answers(
        self, ctx: Context, ev: QuestionAnsweredEvent
    ) -> CollectedAnswersEvent:
        num_to_collect = await ctx.get("num_to_collect")
        results = ctx.collect_events(ev, [QuestionAnsweredEvent] * num_to_collect)
        if results is None:
            return None

        combined_answers = "\n".join([result.model_dump_json() for result in results])
        # save combined_answers to file
        with open(
            f"{self.output_dir}/workflow_output/combined_answers.jsonl", "w"
        ) as f:
            f.write(combined_answers)

        return CollectedAnswersEvent(combined_answers=combined_answers)

    @step
    async def generate_output(
        self, ctx: Context, ev: CollectedAnswersEvent
    ) -> StopEvent:
        output_template = await ctx.get("output_template")
        output_template = "\n".join(
            [doc.get_content("none") for doc in output_template]
        )

        if self._verbose:
            ctx.write_event_to_stream(LogEvent(msg=">> GENERATING FINAL OUTPUT"))

        resp = await self.llm.astream(
            self.generate_output_prompt,
            output_template=output_template,
            answers=ev.combined_answers,
        )

        final_output = ""
        async for r in resp:
            ctx.write_event_to_stream(LogEvent(msg=r, delta=True))
            final_output += r

        # save final_output to file
        with open(f"{self.output_dir}/workflow_output/final_output.md", "w") as f:
            f.write(final_output)

        return StopEvent(result=final_output)

In [44]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")
workflow = RFPWorkflow(
    tools,
    parser=parser,
    llm=llm,
    verbose=True,
    timeout=None,  # don't worry about timeout to make sure it completes
)

## Run the Workflow

Let's run the full workflow and generate the output! 

This will take 5-20 minutes to run and complete. You can inspect the intermediate verbose outputs below as the intermediate questions/answers are generated. The response is streamed back to the user at the end - the response itself is quite long so will take a while to complete! You can also integrate with an observability provider like LlamaTrace/Arize Phoenix in order to view the results.

In [46]:
handler = workflow.run(rfp_template_path="data/jedi_cloud_rfp.pdf")
async for event in handler.stream_events():
    # process request for human input
    if isinstance(event, InputRequiredEvent):
        response = input("Do you like this answer? ")
        response_event = HumanResponseEvent(
            response=response,
            question=event.question,
            answer=event.answer,
            prev_history=event.prev_history
        )
        handler.ctx.send_event(response_event)

    # process logs
    if isinstance(event, LogEvent):
        if event.delta:
            print(event.msg, end="")
        else:
            print(event.msg)

response = await handler
print(str(response))

Running step parse_output_template
Step parse_output_template produced event OutputTemplateEvent
Running step extract_questions
Step extract_questions produced no event
Running step handle_question
Running step handle_question
Running step handle_question
Running step handle_question
Step handle_question produced event InputRequiredEvent
Running step handle_question
>> Asked question: What are the specific requirements for the Transfer Cross Domain Solution to ensure secure data transfer as per the 2018 Raise the Bar Cross Domain Solution Design and Implementation Requirements?
>> Got response: The specific requirements for the Transfer Cross Domain Solution to ensure secure data transfer as per the 2018 Raise the Bar Cross Domain Solution Design and Implementation Requirements are not detailed in the provided document. The document focuses on Microsoft Azure Government's secure and compliant cloud services for U.S. government entities, but it does not specifically address the 2018 Rai

Do you like this answer?  yes


Running step process_human_response
Step process_human_response produced event QuestionAnsweredEvent
>> Processing human response:
Feedback: yes
Original question: What are the specific requirements for the Transfer Cross Domain Solution to ensure secure data transfer as per the 2018 Raise the Bar Cross Domain Solution Design and Implementation Requirements?
Original answer: The specific requirements for the Transfer Cross Domain Solution to ensure secure data transfer as per the 2018 Raise the Bar Cross Domain Solution Design and Implementation Requirements are not detailed in the provided document. The document focuses on Microsoft Azure Government's secure and compliant cloud services for U.S. government entities, but it does not specifically address the 2018 Raise the Bar requirements for Cross Domain Solutions. For precise details on these requirements, it would be best to refer directly to the 2018 Raise the Bar documentation or related official guidelines.
>> Finalized.
Running 

Do you like this answer?  yes


Running step process_human_response
Step process_human_response produced event QuestionAnsweredEvent
>> Processing human response:
Feedback: yes
Original question: What are the specific requirements for the ruggedized, portable compute and storage devices under Category One, and how should they be designed to meet military operational needs?
Original answer: The document "azure_gov.pdf" does not provide specific details about the requirements for ruggedized, portable compute and storage devices under Category One for military operational needs. It primarily focuses on Microsoft Azure Government's secure and compliant cloud services tailored for U.S. government entities, including compliance standards and digital transformation.

For specific requirements related to ruggedized, portable compute and storage devices for military use, it would be necessary to refer to military or defense-specific documentation or standards that outline such requirements. These might include specifications 

...

Running step process_human_response
Step process_human_response produced event QuestionAnsweredEvent
>> Processing human response:
Feedback: yes
Original question: What are the requirements for the Portability Plan and Portability Test, and how should they be demonstrated?
Original answer: I encountered an error while trying to retrieve specific information about the requirements for the Portability Plan and Portability Test, as well as how they should be demonstrated from the Azure Government document. However, I can provide some general guidance based on typical practices.

### General Requirements for Portability Plan and Portability Test:
1. **Portability Plan**:
   - **Documentation**: A detailed plan outlining the steps and processes to ensure data and application portability across different cloud environments.
   - **Compatibility**: Ensure that applications and data formats are compatible with multiple cloud platforms.
   - **Compliance**: Adhere to relevant compliance and reg

Do you like this answer?  yes


Running step process_human_response
Step process_human_response produced event QuestionAnsweredEvent
>> Processing human response:
Feedback: yes
Original question: What are the specific risk management and quality assurance strategies outlined in the JEDI Cloud program documentation?
Original answer: It seems there is an issue with retrieving the information from the document. Unfortunately, without access to the specific document, I can't provide the detailed risk management and quality assurance strategies for the JEDI Cloud program.

However, typically, such programs involve comprehensive risk management strategies that include identifying potential risks, assessing their impact, and implementing mitigation plans. Quality assurance would involve setting standards and processes to ensure the services meet the required specifications and performance criteria.

For precise details, I recommend checking the official documentation or announcements related to the JEDI Cloud program from M

Do you like this answer?  yes


Running step process_human_response
Step process_human_response produced event QuestionAnsweredEvent
>> Processing human response:
Feedback: yes
Original question: What are the specific logical isolation architecture and encryption requirements as defined in the RFP for both classified and unclassified offerings?
Original answer: The specific logical isolation architecture and encryption requirements for classified and unclassified offerings in Microsoft Azure Government are not explicitly detailed in the retrieved documents. However, here are some general insights based on the available information:

1. **Logical Isolation Architecture**:
   - Azure Government provides a physically isolated instance of Microsoft Azure, which includes a dedicated network and geo-replication between locations. This setup ensures that government data is kept separate from commercial data, providing a secure environment for both classified and unclassified offerings.

2. **Encryption Requirements**:
   - 